# Word Problem: Load Distribution

**Objective function**:


$Min \hspace{1mm} R_{x} = \sum_{w=1}^{W} Y_{xw}E_{xw}$

$Y_{xw}$ - activation variable depending on the node and job

$E_{xw}$ - execution time variable depending on the node and job

_**Note**_:
Should have the following csv file inside the folder where this ipynb notebook is located:
- loadDistribution_10n.csv
- loadDistribution_10n_7j.csv
- loadDistribution_50j.csv
- loadDistribution.csv

In [1]:
'''
Import necessary packages
'''
import random
import copy
import math
import queue
import numpy as np
import pandas as pd

In [2]:
'''
Load the loadDistribution dataset
'''
# loading dataset with 10 nodes and 10 jobs
tenData = pd.read_csv('loadDistribution_10n.csv')
header = ['job'+ str(x+1) for x in range(len(tenData.iloc[0,:]))]
data10 = pd.read_csv('loadDistribution_10n.csv', names=header)

# loading dataset with 10 nodes and 7 jobs
smjData = pd.read_csv('loadDistribution_10n_7j.csv')
header = ['job'+ str(x+1) for x in range(len(smjData.iloc[0,:]))]
dataJ = pd.read_csv('loadDistribution_10n_7j.csv', names=header)

# loading dataset with 7 nodes and 50 jobs
largeData = pd.read_csv('loadDistribution_50j.csv')
header = ['job'+ str(x+1) for x in range(len(largeData.iloc[0,:]))]
dataL = pd.read_csv('loadDistribution_50j.csv', names=header)

# loading default dataset with 7 nodes and 10 jobs
defaultData = pd.read_csv('loadDistribution.csv')
header = ['job'+ str(x+1) for x in range(len(defaultData.iloc[0,:]))]
gData = pd.read_csv('loadDistribution.csv', names=header)

In [3]:
gData

,job1,job2,job3,job4,job5,job6,job7,job8,job9,job10
0,10,12,15,13,20,17,33,26,16,21
1,21,20,28,27,35,36,60,48,33,40
2,8,14,14,16,23,20,30,24,18,19
3,15,17,20,17,32,27,47,34,24,32
4,12,12,16,21,15,20,25,32,18,18
5,18,26,22,24,42,36,66,45,32,44
6,17,16,22,18,22,18,32,26,16,20


In [4]:
dataL

,job1,job2,job3,job4,job5,job6,job7,job8,job9,job10,...,job41,job42,job43,job44,job45,job46,job47,job48,job49,job50
0,10,12,15,13,20,17,33,26,16,21,...,10,12,15,13,20,17,33,26,16,21
1,21,20,28,27,35,36,60,48,33,40,...,21,20,28,27,35,36,60,48,33,40
2,8,14,14,16,23,20,30,24,18,19,...,8,14,14,16,23,20,30,24,18,19
3,15,17,20,17,32,27,47,34,24,32,...,15,17,20,17,32,27,47,34,24,32
4,12,12,16,21,15,20,25,32,18,18,...,12,12,16,21,15,20,25,32,18,18
5,18,26,22,24,42,36,66,45,32,44,...,18,26,22,24,42,36,66,45,32,44
6,17,16,22,18,22,18,32,26,16,20,...,17,16,22,18,22,18,32,26,16,20


In [5]:
dataJ

,job1,job2,job3,job4,job5,job6,job7
0,10,12,15,13,20,26,16
1,21,20,28,27,35,48,33
2,8,14,14,16,23,24,18
3,15,17,20,17,32,34,24
4,12,12,16,21,15,32,18
5,18,26,22,24,42,45,32
6,17,16,22,18,22,26,16
7,8,14,14,16,23,24,18
8,15,17,20,17,32,34,24
9,12,12,16,21,15,32,18


In [6]:
data10

,job1,job2,job3,job4,job5,job6,job7,job8,job9,job10
0,10,12,15,13,20,17,33,26,16,21
1,21,20,28,27,35,36,60,48,33,40
2,8,14,14,16,23,20,30,24,18,19
3,15,17,20,17,32,27,47,34,24,32
4,12,12,16,21,15,20,25,32,18,18
5,18,26,22,24,42,36,66,45,32,44
6,17,16,22,18,22,18,32,26,16,20
7,8,14,14,16,23,20,30,24,18,19
8,15,17,20,17,32,27,47,34,24,32
9,12,12,16,21,15,20,25,32,18,18


## Create node class

In [7]:
'''
Create node class which records the node's number
The object created has the total runtime property for fitness comparison
'''
class Node:
    def __init__(self, no):
        self.no = no
        self.jobs = []
        self.tRuntime = 0

In [8]:
'''
Create a class solution which contains the necessary information for a solution,
i.e. fitness, runtime, list of nodes, print function.
'''
class solution:
    nodesList = []
    
    def updateNode(self, sol, data):
        idx = 0
        for i in sol:
            self.nodesList[i].tRuntime += data.iloc[i, idx]
            self.nodesList[i].jobs.append(idx)
            idx += 1
        
    def __init__(self, sol, data):
        self.solution = sol
        self.fitness = 0
        self.nodesList = [Node(x) for x in range(len(data.iloc[:,0]))]
        self.updateNode(self.solution, data)
        
    def updateFitness(self):
        for n in self.nodesList:
            if n.tRuntime > self.fitness:
                # makes sure that the highest runtime among the nodes is taken
                self.fitness = copy.deepcopy(n.tRuntime)
                
    def show(self):
        print("Solution output: ", self.solution)
        for n in self.nodesList:
            print("Node: {}, Jobs: {}".format(n.no + 1, n.jobs))
        print("Total Runtime: ", self.fitness)

## Genetic Algorithm Application

In [9]:
'''
Create random solution:
- values indicates the node to be used
- index determine the job
input: s <-- size of the list
output: [0,1,2,3,4,5,6] <-- sample output
'''
def randomSolution(s, data):
    temp = []
    for i in range(s):
        temp.append(random.randrange(len(data.iloc[:,0])))
        
    return temp

### Genetic Algorithm

#### _Fitness Function_

In [10]:
'''
Fitness function for GA tries to minimize the possible total runtime
of the solutions provided.
input: solutionX, and solutionY <-- list of node numbers, index specify the task
output: True or False
'''
def fitnessGA(solX, solY):
    if solX.fitness < solY.fitness:
        return True
    else:
        return False

In [11]:
'''
This function compares the different solutions inside the population.
It is sorted descending by each solution's fitness value, i.e. total runtime
input: population
output: sorted population based on total runtime
'''
def solComp(pop):
    return sorted(pop, key=lambda x: x.fitness, reverse=True)

In [12]:
'''
Function for selecting parents for breeding.
Only the bottom 80% of the population will be considered for breeding.
input: sorted population
output: index of solution in the population
'''
def selectParent(pop):
    size = int(len(pop) * 0.80)
    rIdx = random.randrange(size)
    return pop[rIdx]

In [13]:
'''
Function for breeding new children (solution) with crossover point
at the middle section.
input: parent 1, parent 2 (both are objects of solution class)
output: child 1, child 2 (both are objects of solution class)
'''
def crossover(p1, p2, data, cp=None):
    cTemp1 = []
    cTemp2 = []
    if cp == None:
        crossPoint = int(len(p1.solution) / 2)
    else:
        crossPoint = cp
    
    cTemp1 = p1.solution[:crossPoint] + p2.solution[crossPoint:]
    cTemp2 = p2.solution[:crossPoint] + p1.solution[crossPoint:]
    
    c1 = solution(cTemp1, data)
    c2 = solution(cTemp2, data)
    
    return c1, c2

In [14]:
'''
Function to mutate child with a 40% chance to mutate
the node working on that index/task.
The probability of the child mutating is also found
within the geneticAlgo()
input: child (Solution object)
output: mutated child (new Solution object)
'''
def mutate(child, data):
    temp = []
    for i in child.solution:
        if random.uniform(0.0,1.0) > 0.60:
            i = random.randrange(len(data.iloc[:,0]))
        temp.append(i)
        
    return solution(temp, data)

In [15]:
def geneticAlgo(pops=30, gen=25, crosspoint=None, cm1_uBound=0.80, cm2_uBound=0.69, data=gData):
    solSize = len(data.iloc[0,:])
    popSize = pops
    topPoint = int(popSize * 0.8)
    cutOff = int(popSize * 0.2)
    popList = []
    generation = gen
    best = 0
    
    for i in range(popSize):
        popList.append(solution(randomSolution(solSize,data), data))
        
        
    sGA = popList[random.randrange(popSize-1)]
    sGA.updateFitness()
    
    bestGA = copy.deepcopy(sGA)
    # start checking per generation
    for i in reversed(range(generation)):
        for p in popList:
            p.updateFitness()
            if fitnessGA(p, bestGA) == True:
                bestGA = copy.deepcopy(p)
    
        qList = []
        
        # sort population based on their values
        popList = solComp(popList)
        
        for i in popList[topPoint:]:
            if not i.fitness:
                qList.append(i)
                
        for i in popList[cutOff:topPoint]:
            qList.append(i)
            
        # breeding
        while len(qList) < popSize:
            parent1 = selectParent(popList)
            parent2 = selectParent(popList)
            while parent1.solution == parent2.solution:
                parent2 = selectParent(popList)
                
            child1, child2 = crossover(parent1, parent2, data, crosspoint)
            
            # randomized upperbound for chance of a child mutating
            if random.uniform(0.0, 1.0) > cm1_uBound:
                child1 = mutate(child1, data)
                
            # second child having higher chances of mutating
            if random.uniform(0.0, 1.0) > cm2_uBound:
                child2 = mutate(child2, data)
                
            qList.append(child1)
            qList.append(child2)
            
        popList = copy.deepcopy(qList)
        
    bestGA.show()

In [16]:
%%time
geneticAlgo()

Solution output:  [0, 2, 6, 0, 6, 4, 4, 3, 2, 1]
Node: 1, Jobs: [0, 3]
Node: 2, Jobs: [9]
Node: 3, Jobs: [1, 8]
Node: 4, Jobs: [7]
Node: 5, Jobs: [5, 6]
Node: 6, Jobs: []
Node: 7, Jobs: [2, 4]
Total Runtime:  45
Wall time: 323 ms


In [17]:
%%time
geneticAlgo(data=dataL)

Solution output:  [3, 4, 3, 1, 2, 5, 6, 6, 4, 5, 2, 1, 3, 2, 2, 4, 5, 4, 4, 2, 6, 6, 1, 0, 2, 2, 6, 0, 0, 4, 3, 0, 6, 4, 4, 2, 0, 6, 1, 2, 0, 2, 0, 0, 3, 0, 3, 1, 1, 6]
Node: 1, Jobs: [23, 27, 28, 31, 36, 40, 42, 43, 45]
Node: 2, Jobs: [3, 11, 22, 38, 47, 48]
Node: 3, Jobs: [4, 10, 13, 14, 19, 24, 25, 35, 39, 41]
Node: 4, Jobs: [0, 2, 12, 30, 44, 46]
Node: 5, Jobs: [1, 8, 15, 17, 18, 29, 33, 34]
Node: 6, Jobs: [5, 9, 16]
Node: 7, Jobs: [6, 7, 20, 21, 26, 32, 37, 49]
Total Runtime:  191
Wall time: 655 ms


In [18]:
%%time
geneticAlgo(data=dataJ)

Solution output:  [7, 1, 2, 0, 4, 6, 8]
Node: 1, Jobs: [3]
Node: 2, Jobs: [1]
Node: 3, Jobs: [2]
Node: 4, Jobs: []
Node: 5, Jobs: [4]
Node: 6, Jobs: []
Node: 7, Jobs: [5]
Node: 8, Jobs: [0]
Node: 9, Jobs: [6]
Node: 10, Jobs: []
Total Runtime:  26
Wall time: 278 ms


In [19]:
%%time
geneticAlgo(data=data10)

Solution output:  [6, 6, 3, 2, 9, 8, 0, 7, 2, 7]
Node: 1, Jobs: [6]
Node: 2, Jobs: []
Node: 3, Jobs: [3, 8]
Node: 4, Jobs: [2]
Node: 5, Jobs: []
Node: 6, Jobs: []
Node: 7, Jobs: [0, 1]
Node: 8, Jobs: [7, 9]
Node: 9, Jobs: [5]
Node: 10, Jobs: [4]
Total Runtime:  43
Wall time: 329 ms


### Comparing optimal output of GA after 5 runs
#### Testing different dataset

In [35]:
%%time
for i in range(5):
    geneticAlgo()

Solution output:  [1, 6, 6, 3, 0, 5, 4, 2, 4, 0]
Node: 1, Jobs: [4, 9]
Node: 2, Jobs: [0]
Node: 3, Jobs: [7]
Node: 4, Jobs: [3]
Node: 5, Jobs: [6, 8]
Node: 6, Jobs: [5]
Node: 7, Jobs: [1, 2]
Total Runtime:  43
Solution output:  [2, 2, 6, 1, 2, 5, 4, 0, 6, 3]
Node: 1, Jobs: [7]
Node: 2, Jobs: [3]
Node: 3, Jobs: [0, 1, 4]
Node: 4, Jobs: [9]
Node: 5, Jobs: [6]
Node: 6, Jobs: [5]
Node: 7, Jobs: [2, 8]
Total Runtime:  45
Solution output:  [2, 2, 2, 0, 6, 3, 4, 0, 5, 4]
Node: 1, Jobs: [3, 7]
Node: 2, Jobs: []
Node: 3, Jobs: [0, 1, 2]
Node: 4, Jobs: [5]
Node: 5, Jobs: [6, 9]
Node: 6, Jobs: [8]
Node: 7, Jobs: [4]
Total Runtime:  43
Solution output:  [0, 4, 2, 3, 4, 3, 4, 0, 2, 1]
Node: 1, Jobs: [0, 7]
Node: 2, Jobs: [9]
Node: 3, Jobs: [2, 8]
Node: 4, Jobs: [3, 5]
Node: 5, Jobs: [1, 4, 6]
Node: 6, Jobs: []
Node: 7, Jobs: []
Total Runtime:  52
Solution output:  [0, 1, 1, 2, 5, 2, 0, 6, 6, 3]
Node: 1, Jobs: [0, 6]
Node: 2, Jobs: [1, 2]
Node: 3, Jobs: [3, 5]
Node: 4, Jobs: [9]
Node: 5, Jobs: []
No

In [36]:
%%time
for i in range(5):
    geneticAlgo(data=dataL)

Solution output:  [3, 3, 5, 0, 2, 6, 0, 6, 1, 6, 4, 0, 4, 4, 3, 4, 3, 1, 0, 2, 6, 5, 4, 0, 0, 2, 4, 5, 2, 5, 2, 5, 0, 5, 0, 3, 4, 2, 3, 0, 4, 2, 5, 2, 2, 1, 3, 1, 0, 0]
Node: 1, Jobs: [3, 6, 11, 18, 23, 24, 32, 34, 39, 48, 49]
Node: 2, Jobs: [8, 17, 45, 47]
Node: 3, Jobs: [4, 19, 25, 28, 30, 37, 41, 43, 44]
Node: 4, Jobs: [0, 1, 14, 16, 35, 38, 46]
Node: 5, Jobs: [10, 12, 13, 15, 22, 26, 36, 40]
Node: 6, Jobs: [2, 21, 27, 29, 31, 33, 42]
Node: 7, Jobs: [5, 7, 9, 20]
Total Runtime:  209
Solution output:  [0, 0, 1, 4, 6, 4, 2, 3, 4, 2, 5, 0, 3, 6, 0, 3, 1, 6, 4, 2, 0, 2, 4, 2, 2, 6, 4, 2, 4, 0, 2, 4, 5, 2, 4, 0, 4, 1, 3, 6, 0, 3, 3, 6, 1, 3, 5, 2, 2, 0]
Node: 1, Jobs: [0, 1, 11, 14, 20, 29, 35, 40, 49]
Node: 2, Jobs: [2, 16, 37, 44]
Node: 3, Jobs: [6, 9, 19, 21, 23, 24, 27, 30, 33, 47, 48]
Node: 4, Jobs: [7, 12, 15, 38, 41, 42, 45]
Node: 5, Jobs: [3, 5, 8, 18, 22, 26, 28, 31, 34, 36]
Node: 6, Jobs: [10, 32, 46]
Node: 7, Jobs: [4, 13, 17, 25, 39, 43]
Total Runtime:  211
Solution output:  

In [37]:
%%time
for i in range(5):
    geneticAlgo(data=dataJ)

Solution output:  [3, 8, 5, 0, 6, 2, 9]
Node: 1, Jobs: [3]
Node: 2, Jobs: []
Node: 3, Jobs: [5]
Node: 4, Jobs: [0]
Node: 5, Jobs: []
Node: 6, Jobs: [2]
Node: 7, Jobs: [4]
Node: 8, Jobs: []
Node: 9, Jobs: [1]
Node: 10, Jobs: [6]
Total Runtime:  24
Solution output:  [2, 5, 3, 2, 7, 0, 9]
Node: 1, Jobs: [5]
Node: 2, Jobs: []
Node: 3, Jobs: [0, 3]
Node: 4, Jobs: [2]
Node: 5, Jobs: []
Node: 6, Jobs: [1]
Node: 7, Jobs: []
Node: 8, Jobs: [4]
Node: 9, Jobs: []
Node: 10, Jobs: [6]
Total Runtime:  26
Solution output:  [2, 9, 1, 3, 6, 7, 2]
Node: 1, Jobs: []
Node: 2, Jobs: [2]
Node: 3, Jobs: [0, 6]
Node: 4, Jobs: [3]
Node: 5, Jobs: []
Node: 6, Jobs: []
Node: 7, Jobs: [4]
Node: 8, Jobs: [5]
Node: 9, Jobs: []
Node: 10, Jobs: [1]
Total Runtime:  28
Solution output:  [8, 1, 9, 6, 2, 7, 0]
Node: 1, Jobs: [6]
Node: 2, Jobs: [1]
Node: 3, Jobs: [4]
Node: 4, Jobs: []
Node: 5, Jobs: []
Node: 6, Jobs: []
Node: 7, Jobs: [3]
Node: 8, Jobs: [5]
Node: 9, Jobs: [0]
Node: 10, Jobs: [2]
Total Runtime:  24
Solution

In [38]:
%%time
for i in range(5):
    geneticAlgo(data=data10)

Solution output:  [6, 7, 5, 1, 3, 2, 9, 2, 4, 6]
Node: 1, Jobs: []
Node: 2, Jobs: [3]
Node: 3, Jobs: [5, 7]
Node: 4, Jobs: [4]
Node: 5, Jobs: [8]
Node: 6, Jobs: [2]
Node: 7, Jobs: [0, 9]
Node: 8, Jobs: [1]
Node: 9, Jobs: []
Node: 10, Jobs: [6]
Total Runtime:  44
Solution output:  [0, 2, 4, 5, 6, 9, 7, 8, 0, 4]
Node: 1, Jobs: [0, 8]
Node: 2, Jobs: []
Node: 3, Jobs: [1]
Node: 4, Jobs: []
Node: 5, Jobs: [2, 9]
Node: 6, Jobs: [3]
Node: 7, Jobs: [4]
Node: 8, Jobs: [6]
Node: 9, Jobs: [7]
Node: 10, Jobs: [5]
Total Runtime:  34
Solution output:  [2, 7, 7, 7, 0, 1, 4, 6, 9, 3]
Node: 1, Jobs: [4]
Node: 2, Jobs: [5]
Node: 3, Jobs: [0]
Node: 4, Jobs: [9]
Node: 5, Jobs: [6]
Node: 6, Jobs: []
Node: 7, Jobs: [7]
Node: 8, Jobs: [1, 2, 3]
Node: 9, Jobs: []
Node: 10, Jobs: [8]
Total Runtime:  44
Solution output:  [8, 7, 7, 6, 6, 5, 2, 3, 8, 0]
Node: 1, Jobs: [9]
Node: 2, Jobs: []
Node: 3, Jobs: [6]
Node: 4, Jobs: [7]
Node: 5, Jobs: []
Node: 6, Jobs: [5]
Node: 7, Jobs: [3, 4]
Node: 8, Jobs: [1, 2]
Node: 

--------------------------------
--------------------------------

# Testing

### Testing different population sizes

In [20]:
%%time
geneticAlgo(pops=50)

Solution output:  [5, 3, 3, 6, 2, 1, 4, 6, 4, 0]
Node: 1, Jobs: [9]
Node: 2, Jobs: [5]
Node: 3, Jobs: [4]
Node: 4, Jobs: [1, 2]
Node: 5, Jobs: [6, 8]
Node: 6, Jobs: [0]
Node: 7, Jobs: [3, 7]
Total Runtime:  44
Wall time: 452 ms


In [21]:
%%time
geneticAlgo(pops=70)

Solution output:  [2, 4, 2, 6, 5, 1, 4, 6, 0, 0]
Node: 1, Jobs: [8, 9]
Node: 2, Jobs: [5]
Node: 3, Jobs: [0, 2]
Node: 4, Jobs: []
Node: 5, Jobs: [1, 6]
Node: 6, Jobs: [4]
Node: 7, Jobs: [3, 7]
Total Runtime:  44
Wall time: 603 ms


In [22]:
%%time
geneticAlgo(pops=100)

Solution output:  [2, 6, 2, 5, 0, 3, 4, 6, 1, 2]
Node: 1, Jobs: [4]
Node: 2, Jobs: [8]
Node: 3, Jobs: [0, 2, 9]
Node: 4, Jobs: [5]
Node: 5, Jobs: [6]
Node: 6, Jobs: [3]
Node: 7, Jobs: [1, 7]
Total Runtime:  42
Wall time: 877 ms


### Testing different generations

In [23]:
%%time
geneticAlgo(gen=50)

Solution output:  [2, 5, 1, 0, 2, 4, 4, 0, 3, 6]
Node: 1, Jobs: [3, 7]
Node: 2, Jobs: [2]
Node: 3, Jobs: [0, 4]
Node: 4, Jobs: [8]
Node: 5, Jobs: [5, 6]
Node: 6, Jobs: [1]
Node: 7, Jobs: [9]
Total Runtime:  45
Wall time: 519 ms


In [24]:
%%time
geneticAlgo(gen=70)

Solution output:  [2, 0, 3, 6, 5, 0, 2, 4, 1, 6]
Node: 1, Jobs: [1, 5]
Node: 2, Jobs: [8]
Node: 3, Jobs: [0, 6]
Node: 4, Jobs: [2]
Node: 5, Jobs: [7]
Node: 6, Jobs: [4]
Node: 7, Jobs: [3, 9]
Total Runtime:  42
Wall time: 637 ms


In [25]:
%%time
geneticAlgo(gen=1000)

Solution output:  [0, 5, 1, 6, 2, 3, 4, 6, 0, 2]
Node: 1, Jobs: [0, 8]
Node: 2, Jobs: [2]
Node: 3, Jobs: [4, 9]
Node: 4, Jobs: [5]
Node: 5, Jobs: [6]
Node: 6, Jobs: [1]
Node: 7, Jobs: [3, 7]
Total Runtime:  44
Wall time: 9.7 s


### Testing different crossover point

In [26]:
%%time
geneticAlgo(crosspoint=3)

Solution output:  [0, 0, 2, 4, 5, 6, 4, 3, 2, 1]
Node: 1, Jobs: [0, 1]
Node: 2, Jobs: [9]
Node: 3, Jobs: [2, 8]
Node: 4, Jobs: [7]
Node: 5, Jobs: [3, 6]
Node: 6, Jobs: [4]
Node: 7, Jobs: [5]
Total Runtime:  46
Wall time: 280 ms


In [27]:
%%time
geneticAlgo(crosspoint=7)

Solution output:  [2, 4, 6, 2, 5, 0, 3, 1, 6, 2]
Node: 1, Jobs: [5]
Node: 2, Jobs: [7]
Node: 3, Jobs: [0, 3, 9]
Node: 4, Jobs: [6]
Node: 5, Jobs: [1]
Node: 6, Jobs: [4]
Node: 7, Jobs: [2, 8]
Total Runtime:  48
Wall time: 270 ms


In [28]:
%%time
geneticAlgo(crosspoint=8)

Solution output:  [4, 4, 3, 0, 2, 5, 6, 1, 6, 2]
Node: 1, Jobs: [3]
Node: 2, Jobs: [7]
Node: 3, Jobs: [4, 9]
Node: 4, Jobs: [2]
Node: 5, Jobs: [0, 1]
Node: 6, Jobs: [5]
Node: 7, Jobs: [6, 8]
Total Runtime:  48
Wall time: 400 ms


### Testing different mutation probability

In [29]:
%%time
geneticAlgo(cm1_uBound=0.50, cm2_uBound=0.50)

Solution output:  [1, 3, 4, 1, 2, 6, 4, 0, 0, 5]
Node: 1, Jobs: [7, 8]
Node: 2, Jobs: [0, 3]
Node: 3, Jobs: [4]
Node: 4, Jobs: [1]
Node: 5, Jobs: [2, 6]
Node: 6, Jobs: [9]
Node: 7, Jobs: [5]
Total Runtime:  48
Wall time: 321 ms


In [30]:
%%time
geneticAlgo(cm1_uBound=0.30, cm2_uBound=0.70)

Solution output:  [6, 5, 4, 0, 6, 4, 2, 0, 1, 3]
Node: 1, Jobs: [3, 7]
Node: 2, Jobs: [8]
Node: 3, Jobs: [6]
Node: 4, Jobs: [9]
Node: 5, Jobs: [2, 5]
Node: 6, Jobs: [1]
Node: 7, Jobs: [0, 4]
Total Runtime:  39
Wall time: 321 ms


In [31]:
%%time
geneticAlgo(cm1_uBound=0.20, cm2_uBound=0.20)

Solution output:  [3, 4, 5, 5, 6, 3, 0, 2, 1, 6]
Node: 1, Jobs: [6]
Node: 2, Jobs: [8]
Node: 3, Jobs: [7]
Node: 4, Jobs: [0, 5]
Node: 5, Jobs: [1]
Node: 6, Jobs: [2, 3]
Node: 7, Jobs: [4, 9]
Total Runtime:  46
Wall time: 343 ms


### Testing different population sizes and generation

In [32]:
%%time
geneticAlgo(pops=50, gen=50)

Solution output:  [5, 2, 5, 3, 2, 6, 4, 0, 0, 4]
Node: 1, Jobs: [7, 8]
Node: 2, Jobs: []
Node: 3, Jobs: [1, 4]
Node: 4, Jobs: [3]
Node: 5, Jobs: [6, 9]
Node: 6, Jobs: [0, 2]
Node: 7, Jobs: [5]
Total Runtime:  43
Wall time: 804 ms


In [33]:
%%time
geneticAlgo(pops=70, gen=70)

Solution output:  [1, 1, 5, 2, 0, 4, 2, 6, 3, 6]
Node: 1, Jobs: [4]
Node: 2, Jobs: [0, 1]
Node: 3, Jobs: [3, 6]
Node: 4, Jobs: [8]
Node: 5, Jobs: [5]
Node: 6, Jobs: [2]
Node: 7, Jobs: [7, 9]
Total Runtime:  46
Wall time: 1.56 s


In [34]:
%%time
geneticAlgo(pops=100, gen=100)

Solution output:  [5, 0, 1, 6, 0, 4, 2, 3, 4, 6]
Node: 1, Jobs: [1, 4]
Node: 2, Jobs: [2]
Node: 3, Jobs: [6]
Node: 4, Jobs: [7]
Node: 5, Jobs: [5, 8]
Node: 6, Jobs: [0]
Node: 7, Jobs: [3, 9]
Total Runtime:  38
Wall time: 3.09 s
